In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [9]:
df = pd.read_parquet('../data/end_df.parquet')

In [10]:
LE = {'place': LabelEncoder(), 'product': LabelEncoder()}
df['place'] = LE['place'].fit_transform(df['place'])
df['product'] = LE['product'].fit_transform(df['product'])

In [20]:
df.corr(numeric_only=True, method='pearson')['Our_company']

place            -0.197688
product           0.050665
Our_company       1.000000
cost              0.861602
Арториас&Co       0.963828
Длань господня    0.934589
ЛилIT             0.934168
Светлые Души      0.928461
hot              -0.010395
rain              0.012648
snow             -0.006228
amount           -0.280229
Name: Our_company, dtype: float64

In [21]:
df

,place,product,date,Our_company,cost,Арториас&Co,Длань господня,ЛилIT,Светлые Души,hot,rain,snow,amount
0,0,0,2216-01-02,3.90,2.07,3.78,4.21,4.18,3.75,1.0,0.0,0.0,39.889286
1,0,0,2216-01-03,3.90,2.07,3.78,4.21,4.18,3.75,1.0,0.0,0.0,42.086108
2,0,0,2216-01-04,3.90,2.07,3.78,4.21,4.18,3.75,0.0,0.0,0.0,37.922631
3,0,0,2216-01-05,3.90,2.07,3.78,4.21,4.18,3.75,0.0,0.0,0.0,38.839745
4,0,0,2216-01-06,3.90,2.07,3.78,4.21,4.18,3.75,0.0,0.0,1.0,45.990511
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,4,2,2218-09-23,9.28,3.26,15.67,14.58,14.02,15.36,0.0,1.0,0.0,54.451677
14996,4,2,2218-09-24,9.28,7.29,15.67,14.58,14.02,15.36,0.0,0.0,0.0,48.684146
14997,4,2,2218-09-25,9.28,7.29,15.67,14.58,14.02,15.36,0.0,0.0,1.0,57.460737
14998,4,2,2218-09-26,9.28,7.29,15.67,14.58,14.02,15.36,0.0,0.0,1.0,55.267398
